# $\mathscr P$ space
OOP description of the space of proteins

In [6]:
class Protein:
    def __init__():
        0
    def mutation(param):
        0

# $H_t : \mathscr P \rightarrow \mathscr P$
Function that takes a protein as input and returns it after heat stress

In [4]:
def H(protein):
    return protein                        # TODO

# $g : \mathscr P \rightarrow \mathscr I$
Function that returns the <b>$\Delta E$</b>

In [5]:
def g(protein):
    return 0.                            # TODO

# Learner
Agent able to learn from experience

In [18]:
import numpy as np
class Learner:
    def __init__(self):
        self.X = np.array([])
        self.Y = np.array([])
    def learn(self,x,y):
        if len(self.X) == 0:
            # first entry
            self.X = (x * 1)[None,:]
            self.Y = y * 1
        else:
            self.X = np.r_[self.X,x[None,:]]
            self.Y = np.r_[self.Y,y]
    def predict(self,x):
        return self.X,self.Y

In [21]:
L = Learner()
L.learn(np.random.randn(2),1)
L.learn(np.random.randn(2),1)
L.learn(np.random.randn(2),2)
L.predict(0)

(array([[1.09687392, 0.70368307],
        [0.24937474, 0.18462425],
        [0.85485559, 0.21770373]]),
 array([1, 1, 2]))

# Guide
Agent who drives the configuration exploration, modulating the trust wrt the Learner

In [22]:
class Guide:
    def __init__(self):
        self.eta = 1.
        
    def trust():
        self.eta *= 1.1          # if I build trust, the "confidence in the learner" increase
        
    def untrust():
        self.eta *= 0.9          # if I disappoint you, otherwise...

        
    def softmax(probabilityDistribution):
        # returns the homotopied probability distribution
        
        return np.exp(probabilityDistribution)** self.eta / (np.exp(probabilityDistribution) ** self.eta).sum()

    def choose(probabilityDistribution):
        # takes as input a probability distribution and "homotopies" it with the uniform distribution
        # according to its trust.
        # returns the extracted index
        
        P = softmax(probabilityDistribution)
        return np.random.choice(len(P),P)

# Team
Class that models the cooperative system between the le